In [1]:
import hashlib
import os

### Register New Users

In [2]:
# Function to hash the password
def hash_password(password):
    return hashlib.sha256(password.encode('utf-8')).hexdigest()

# Function to check if username exists in the file
def unique_username(username, filename="credentials.txt"):
    try:
        with open(filename, 'r') as file:
            for line in file:
                stored_username, _ = line.strip().split(',')
                if stored_username == username:
                    return False  # Username exists
    except FileNotFoundError:
        return True  # File doesn't exist, so username is unique

    return True  # Username is unique if no match found

# Function to store the username and hashed password
def store_user_data(username, password, filename="credentials.txt"):
    if unique_username(username, filename)== False:
        print("Username is already taken. Please choose another.")
        return
    
    # Hash the password before storing
    hashed_password = hash_password(password)
    
    with open(filename, 'a') as file:
        file.write(f"{username},{hashed_password}\n")
    
    print("User data stored successfully.")

# Main function to prompt the user for input and store the data
def user_registration():
    username = input("Username: ")
    password = input("Password: ")
    
    store_user_data(username, password)
    return username

### Validate Logins

In [3]:
# Function to validate the user's credentials
def validate_user(username, password, filename="credentials.txt"):
    try:
        with open(filename, 'r') as file:
            for line in file:
                stored_username, stored_hashed_password = line.strip().split(',')
                if stored_username == username and stored_hashed_password == hash_password(password):
                    return True  # Credentials are valid
    except FileNotFoundError:
        return False  # File doesn't exist, no users found

    return False  # Credentials are invalid if no match found

# Function to simulate task manager access
def task_manager():
    print("Welcome to the Task Manager!")
    # Here, you can add further code for task management functionality

# Function to prompt the user for their credentials
def user_login():
    username = input("Username: ")
    password = input("Password: ")

    if validate_user(username, password):
        print("Login successful!")
    else:
        print("Invalid username or password. Please try again.")
    return username

### Add Tasks

In [4]:
# Function to generate a unique task ID
def generate_task_id(filename="tasks.txt"):
    task_id = 1
    if os.path.exists(filename):
        # Read the file and get the last task ID
        with open(filename, 'r') as file:
            lines = file.readlines()
            if lines:
                last_line = lines[-1]
                last_task_id = int(last_line.split(',')[1])
                task_id = last_task_id + 1
    return task_id

# Function to store the task
def store_task(description, username, filename="tasks.txt"):
    task_id = generate_task_id(filename)  # Generate a unique task ID
    status = "Pending"  # Default status for new tasks
    
    # Save the task to the file
    with open(filename, 'a') as file:
        file.write(f"{username},{task_id},{description},{status}\n")
    
    print(f"Task added successfully! Task ID: {task_id}, Status: {status}")

# Function to prompt the user for task description
def add_task(username):
    description = input("Enter the task description: ")
    store_task(description, username)  # Store the task

### Display Tasks

In [5]:
# Function to retrieve and display tasks for the logged-in user
def display_user_tasks(username, filename="tasks.txt"):
    if not os.path.exists(filename):
        print("No tasks available.")
        return
    
    found = False
    with open(filename, 'r') as file:
        print(f"\nTasks for {username}:")
        print(f"{'Task ID':<10}{'Description':<30}{'Status'}")
        print("-" * 50)
        
        for line in file:
            task_data = line.strip().split(',')
            task_username, task_id, description, status = task_data
            
            if task_username == username:
                found = True
                print(f"{task_id:<10}{description:<30}{status}")
        
        if not found:
            print("No tasks found for this user.")

# Example of a prompt for the logged-in user to display their tasks
def view_tasks(username):
    display_user_tasks(username)

### Update Tasks

In [6]:
# Function to update the status of a task to 'Completed'
def update_task_status(task_id, filename="tasks.txt"):
    if not os.path.exists(filename):
        print("No tasks file found.")
        return

    tasks_updated = False
    tasks = []

    # Read the tasks and store them in a list
    with open(filename, 'r') as file:
        for line in file:
            task_data = line.strip().split(',')
            task_username, stored_task_id, description, status = task_data
            tasks.append((task_username, stored_task_id, description, status))

    # Iterate through the tasks and update the status of the selected task
    with open(filename, 'w') as file:
        for task_username, stored_task_id, description, status in tasks:
            if stored_task_id == task_id:
                # Update the task's status to 'Completed'
                status = "Completed"
                tasks_updated = True
            file.write(f"{task_username},{stored_task_id},{description},{status}\n")
    
    if tasks_updated:
        print(f"Task ID {task_id} status updated to Completed.")
    else:
        print(f"Task ID {task_id} not found.")

# Function to prompt the user to select a task ID and update its status
def task_update():
    task_id = input("Enter the Task ID you want to update to 'Completed': ")
    
    # Ensure task ID is a number
    if not task_id.isdigit():
        print("Invalid Task ID. Please enter a valid number.")
        return
    
    update_task_status(task_id)

### Delete Tasks

In [7]:
# Function to delete a task by its ID
def delete_task(task_id, filename="tasks.txt"):
    if not os.path.exists(filename):
        print("No tasks file found.")
        return

    tasks_deleted = False
    tasks = []

    # Read all tasks from the file
    with open(filename, 'r') as file:
        for line in file:
            task_data = line.strip().split(',')
            task_username, stored_task_id, description, status = task_data
            if stored_task_id != task_id:
                tasks.append((task_username, stored_task_id, description, status))  # Keep task if ID doesn't match
            else:
                tasks_deleted = True  # Task is marked for deletion

    # If the task was deleted, write the updated tasks back to the file
    if tasks_deleted:
        with open(filename, 'w') as file:
            for task_username, stored_task_id, description, status in tasks:
                file.write(f"{task_username},{stored_task_id},{description},{status}\n")
        print(f"Task ID {task_id} has been deleted.")
    else:
        print(f"Task ID {task_id} not found.")

# Function to prompt the user for a task ID to delete
def task_deletion():
    task_id = input("Enter the Task ID you want to delete: ")

    # Ensure task ID is a number
    if not task_id.isdigit():
        print("Invalid Task ID. Please enter a valid number.")
        return
    
    delete_task(task_id)

### Menu

In [8]:
# Function to display the menu
def display_menu():
    print("\nTask Manager Menu:")
    print("1. Add a Task")
    print("2. View Tasks")
    print("3. Mark a Task as Completed")
    print("4. Delete a Task")
    print("5. Logout")

# Main function to handle the user session
def task_manager():
    while True:
        print("\nWelcome to the Task Manager.")
        print("1. Signin")
        print("2. Register")
        print("3. Exit")
        
        option = input("Choose an option (1-3): ")

        if option == '1':
            username = user_login()
        elif option == '2':
            username = user_registration()
        elif option == '3':
            print("Exiting the Task Manager.")
            return
        else:
            print("Invalid option. Please choose 1, 2, or 3.")

        while True:
            display_menu()
            choice = input("Select an option (1-5): ")
    
            if choice == '1':
                add_task(username)
            elif choice == '2':
                view_tasks(username)
            elif choice == '3':
                task_update()
            elif choice == '4':
                task_deletion()
            elif choice == '5':
                print("Logging out...")
                break
            else:
                print("Invalid option. Please select a valid option from the menu.")

In [ ]:
# Run the prompt user function
if __name__ == "__main__":
    task_manager()


Welcome to the Task Manager.
1. Signin
2. Register
3. Exit
